In [1]:
import pandas as pd
import numpy as np
import re
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import scipy
from scipy.sparse import csr_matrix
import numpy as np
from sklearn import tree
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [2]:
consumer_key = 'bkK2DrDErBVyDDcH7k58Ky3AW'
consumer_secret = 'sHP8ryCUARGXfZMGicB2PdmiAC5V3uGdxtXi8foTUUe3DDo3r4'
access_token = '774855636383326208-1OUG5LiM3adHpY5INeYd9pvYai8JDvd'
access_token_secret = 'zvJ90xh3v1QOdCs6sdhahpem4fF6BHZHYmbEZN01SOv4X'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
keywords=["disaster",
"relief",
"earthquake",
"flood",
"hurricane",
"storm",
"tsunami",
"cyclone",
"landslide",
"avalanche",
"natural",
"volcano",
"fema",
"fire",
"police",
"shooting", 
"terrorism",
"nuclear", 
"attack", 
"catastrophe", 
"bombs", 
"calamity", 
"accident", 
"tragedy", 
"epidemic" 
"evacuation",
"shaking", 
"aftershock", 
"disaster",
"evacuate", 
"evacuated", 
"tsunami", 
"flooding", 
"volcano", 
"eruption", 
"massive", 
"killed", 
"damage", 
"damaged", 
"destroyed", 
"destroy", 
"explosion",
"blackout",
"blizzard",
"thunderstorm",
"drought",
"fatal",
"fatality",
"hailstorm",
"assault",
"rainstorm",
"snow",
"snowstorm",
"tornado",
"casualty",
"drown"]

In [4]:
month={
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
}

In [5]:
tweetsCsv = pd.read_csv('tweets.csv', encoding = "ISO-8859-1", header=None, usecols=[2, 5])

In [6]:
df = pd.DataFrame(tweetsCsv)    

In [7]:
tweets=[]
for row in zip(df[2], df[5]):
    for keyword in keywords:
        if keyword in row[1]:
            split_date=row[0].split(" ")
            date=split_date[5]+"/"+str(month[split_date[1]])+"/"+split_date[2]
            clean_tweet= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", row[1]).split())
            tweet=(date, clean_tweet)
            tweets.append(tweet)

In [8]:
len(set(tweets))

19346

In [9]:
tweets_from_api=[]
for keyword in keywords:
    tweets_from_api += api.search(q = keyword)

In [10]:
listTuples=[]
for tweet in tweets_from_api: 
    clean_tweet= ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split())
    split_date=tweet._json['created_at'].split(" ")
    date=split_date[5]+"/"+str(month[split_date[1]])+"/"+split_date[2]
    tweetTuple=(date, clean_tweet)
    listTuples+=tweetTuple

In [11]:
listTuples

['2018/11/11',
 'RT If you ve been affected by a wildfire amp need emotional support reach out Call 1 800 985 5990 or text TalkWithUs to 667',
 '2018/11/11',
 'RT Did the president literally take the side of a natural disaster over Americans losing their lives and homes today Yes',
 '2018/11/11',
 'RT A fool and imbecile of such staggering dimensions that it defies description in the English language Americans have d',
 '2018/11/11',
 'RT Hey intercommunal neighbors of the world If you like this check out this thread of ways to support community lead disas',
 '2018/11/11',
 'RT I can t think of a single natural disaster Trump or his administration has competently responded to What a disgrace',
 '2018/11/11',
 'RT Did the president literally take the side of a natural disaster over Americans losing their lives and homes today Yes',
 '2018/11/11',
 'RT I can t think of a single natural disaster Trump or his administration has competently responded to What a disgrace',
 '2018/11/11',
 'Th

In [12]:
allTweets = tweets+listTuples

In [13]:
len(set(allTweets))

19935

In [14]:
allTweets

[('2009/4/06', 'my whole body feels itchy and like its on fire'),
 ('2009/4/06', 'spring break in plain city it s snowing'),
 ('2009/4/06',
  'Broadband plan a massive broken promise via www diigo com tautao Still waiting for broadband we are'),
 ('2009/4/06', 'I m sooo sad they killed off Kutner on House whyyyyyyyy'),
 ('2009/4/06', 'Agh snow'),
 ('2009/4/06',
  'but this is canada canada is weird we re supposed to get snow through wednesday ugh'),
 ('2009/4/06', 'What tragedy and disaster in the news this week'),
 ('2009/4/06', 'What tragedy and disaster in the news this week'),
 ('2009/4/06', 'What tragedy and disaster in the news this week'),
 ('2009/4/06', 'im lonely keep me company 22 female california'),
 ('2009/4/06', 'sooo sick of the snow ughh'),
 ('2009/4/06',
  'Is terrified she accidentally deleted a reference in her management assignment and hopes she doesnt get in trouble when she gets back'),
 ('2009/4/06',
  'still sick feeling a bit better got some new medicine hope I

In [15]:
indexed_tweets=[]
final_tweets=[]
i=1
for t in allTweets:
    today = datetime.date(2009, 4, 4)
    margin = datetime.timedelta(days = 100)
    if t is "" or t is None:
        continue
    sdate = t[0].split("/")
    if len(sdate)<3:
        continue
    if today - margin <= datetime.date(int(sdate[0]), int(sdate[1]), int(sdate[2])) <= today + margin:
        final_tweets.append(t)
        indexed_tweet=(i, t)
        i+=1
        indexed_tweets.append(indexed_tweet)

In [16]:
len(indexed_tweets)

21475

In [17]:
classified_tweets = open('classifiedTweets.txt', 'r')

In [18]:
classified_tweets= classified_tweets.read()

In [19]:
len(classified_tweets)

3123275

In [20]:
classified_tweets=classified_tweets.split("\n")

In [21]:
classified_tweets[100]

'In about a month students would have set their pens ablaze in The Torch Publications\'... http://t.co/9FxPiXQuJt",548349290,1,6.29E+17,FALSE,5,Inang Pamantasan,778244902,finalized,https://www.facebook.com/thetorchpnu/posts/812437058871537,https://www.facebook.com/,\u202aFacebook\u202c,,,,,,'

In [22]:
classified_tweets[105].split(",")[1]+"   "+ classified_tweets[105].split(",")[4]

"Relevant   @Navista7 Steve these fires out here are something else! California is a tinderbox - and this clown was setting my 'hood ablaze @News24680"

In [23]:
training=[]
training_label=[]
training_data=[]
classified_tweets[1].split(",")
for i in range(len(classified_tweets)-1):
    classified_tweet= classified_tweets[i].split(",")
    if len(classified_tweet)<=4:
        continue
    if classified_tweet[1] in ["Relevant", "Not Relevant"]:
        training_data.append(classified_tweet[4])
        label= 1 if classified_tweet[1]=="Relevant" else 0
        training_label.append(label)

In [24]:
t = [final_tweet[1] for final_tweet in final_tweets]
x = list(training_data) + t

In [25]:
test_x=np.asarray(training_data[:int(len(training_data)*0.20)])
training_data= np.asarray(training_data)
training_data.shape
test_x.shape

test_y=np.asarray(training_label[:int(len(training_label)*0.20)])
training_label= np.asarray(training_label)

In [26]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(x)

In [27]:
X_test = vectors[int(len(training_data)*0.90):]
X_train = vectors[:int(len(training_data))]
Y_test = training_label[int(len(training_label)*0.90):]
Y_train = training_label[:X_train.shape[0]]

In [28]:
print(X_test.shape)
print(Y_test.shape)

(22561, 40443)
(1086,)


In [29]:
def get_score(labels):
    return accuracy_score(Y_test, labels, normalize=True)

In [30]:
logreg = linear_model.LogisticRegression()
def get_accuracy():
    countp=0
    countn=0
    global logreg
    logreg = logreg.fit(X_train, Y_train)
    predicted_labels = logreg.predict(X_test)
    i=0
    index_tweets=[]
    for p in predicted_labels:
        if p==1:
            countp+=1
            index_tweets.append(i)
        elif p==0:
            countn+=1
        i+=1
            
    relevant_tweets=[]
    for index in index_tweets:
        relevant_tweets.append(x[index])
    return relevant_tweets
#     return get_score(predicted_labels)

In [31]:
from sklearn import linear_model
get_accuracy()

["I'm on top of the hill and I can see a fire in the woods...",
 "#raining #flooding #Florida #TampaBay #Tampa 18 or 19 days. I've lost count ",
 "What's up man?",
 'London is cool ;)',
 '@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C',
 'We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw',
 'I wanted to set Chicago ablaze with my preaching... But not my hotel! http://t.co/o9qknbfOFX',
 '"Schoolboy ‰\uf88fׂ Aftershock (Original Mix)',
 "@KJForDays I'm seeing them and Issues at aftershock ??",
 '320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/THyzOMVWU0 | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #Ices‰\uf88f_ http://t.co/83jOO0xk29',
 '#WisdomWed BONUS - 5 Minute Daily Habits that could really improve your life. How many do you already do? #lifehacks http://t.co/TBm9FQb8cW',
 '"&gt;&gt; $15 Aftershock : Protect Yourself and Profit in the Next Global Financial... ##book http://t.co/f6ntUc734Z',
 'Aftershock https://t.co/Ecy4U623nO',
 "Sometimes you face dif

In [32]:
rel_tweets= get_accuracy()
vectors = vectorizer.fit_transform(rel_tweets)
sum_freq = vectors.sum(axis=0)

In [33]:
words_freq = [(word, sum_freq[0,idx]) for word, idx in vectorizer.vocabulary_.items()]

In [34]:
words_freq = sorted(words_freq, key = lambda x:x[1], reverse= True)

In [35]:
final_rel_words=[]
for x,y in words_freq:
    if x in keywords:
        rel_word= [x,y]
        final_rel_words.append(rel_word)

In [36]:
print(len(rel_tweets))

2517


In [37]:
c = len(final_rel_words)
result = []
for i in range(c):
    final_rel_words[i] = (final_rel_words[i][0], c-i)
dict_of_words = dict(final_rel_words)
for tweet in rel_tweets:
    score = 0
    words_in_tweet = tweet.split(' ')
    for word in words_in_tweet:
        c = dict_of_words.get(word)
        if c != 0 and c is not None:
            score += dict_of_words.get(word)
        else:
            for key, value in dict_of_words.items(): 
                if key in word:
                    score += value
    result.append(tuple((tweet,score)))
result = list(sorted(result, key = lambda x: x[1], reverse = True))

In [38]:
top_tweets= result[:5]
i=1
message_tweets="Top Tweets:"
for top_tweet in top_tweets:
    message_tweets+="\n"+str(i)+" " + top_tweet[0]
    i+=1

In [39]:
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from twilio.rest import Client

In [40]:
app = Flask(__name__)
account_sid = 'AC927ba002034f4ef9fe90661ee1afce67'
auth_token = '0e549ab8532e6836eae8692d26eb5e3b'
client = Client(account_sid, auth_token)

In [41]:
@app.route("/sms", methods=['GET', 'POST'])
def sms_ahoy_reply():
    #"""Respond to incoming messages with a friendly SMS."""
    # Start our response
    resp = MessagingResponse()
    #print(request.values.get('From'))
    # Add a message
    global message_tweets
    resp.message(message_tweets)
    tmp = ['+18578694005', '+18575448237', '+19789303476']
    for i in tmp:
        message = client.messages \
                .create(
                     body=message_tweets,
                     from_='+18053605253',
                     to=i
                 )
    return str(resp)

In [ ]:
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2018 11:56:36] "POST /sms HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2018 13:40:45] "POST /sms HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2018 13:48:47] "POST /sms HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2018 14:03:05] "POST /sms HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2018 14:22:30] "POST /sms HTTP/1.1" 200 -


In [46]:
message_tweets =message_tweets.split("\n")
for message in message_tweets:
    print(message)


Top Tweets:
1 Udhampur terror attack: Militants attack police post 2 SPOs injured: Suspected militants tonight attacked a p... http://t.co/FPhFESemyJ
2 Udhampur terror attack: Militants attack police post 2 SPOs injured: Suspected militants tonight attacked a p... http://t.co/Cwm0ULqu3E
3 Cop injured in gunfight as militants attack Udhampur police post: Suspected militants attacked a police post i... http://t.co/ERW7FdxnCr
4 Cop injured in gunfight as militants attack Udhampur police post: Suspected militants attacked a police post i... http://t.co/2h0dPMv2Ef
5 natural disaster ‰ׂ News Stories About natural disaster - Page 1 | Newser http://t.co/TB8gZEMbXU
